# USB Standard Descriptors 列舉
https://wwssllabcd.github.io/blog/2012/11/28/usb-emulation/  
https://www.beyondlogic.org/usbnutshell/  

### 需安裝

```
pip install pyusb

```

### Windows DLL
[libusb-win32-devel-filter-1.2.6.0.exe](https://sourceforge.net/projects/libusb-win32/files/libusb-win32-releases/1.2.6.0/)  

https://sourceforge.net/projects/libusb-win32/  



### Linux 權限
https://ubuntuforums.org/showthread.php?t=1682084  
https://linuxconfig.org/tutorial-on-how-to-write-basic-udev-rules-in-linux  
```
If anyone is interested.. fixed with udev rule:

cd /etc/udev/rules.d/

nano 70-ch341.rules

SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", GROUP="dialout"
SUBSYSTEM=="usb", ATTRS{idVendor}=="1a86", MODE="666"

then reloaded udev rules:

$ sudo udevadm control --reload-rules  
$ sudo udevadm trigger

```

In [1]:
%pylab inline  

import usb

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'codes')))

path = os.sep.join(['..', '..', '..', '..', '..', '待處理', 'ORM', 'bitbucket', 'github', 'codes'])
sys.path.append(path) 

path = os.sep.join(['..', '..', '..', '..', '..', '待處理', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'])
sys.path.append(path)

import universal_serial_bus
from universal_serial_bus import * 
from universal_serial_bus.orm.usb20 import *

In [3]:
db_path = 'C:\\Users\\Wei\\Dropbox\\Coding\\notebooks\\專案\\待處理\\Universal Serial Bus\\bitbucket\\github\\spec\\db\\'
db_path = db_path + 'usb_2_0.sqlite'
db_url = 'sqlite:///' + db_path 

In [4]:
from universal_serial_bus.orm import ModelBuilder

ModelBuilder.gen_all(db_url)


class ClassCode(OrmClassBase):

    fields_sizes = [('base_code', 1),]

    def __init__(self, description, descriptor_usage, base_code):

        self.description = description
        self.descriptor_usage = descriptor_usage
        self.base_code = base_code


class DescriptorType(OrmClassBase):

    fields_sizes = [('bDescriptorType', 1),]

    def __init__(self, dscrpt_type, bDescriptorType):

        self.dscrpt_type = dscrpt_type
        self.bDescriptorType = bDescriptorType


class DeviceQualifierDescriptor(OrmClassBase):

    fields_sizes = [('bLength', 1), ('bDescriptorType', 1), ('bcdUSB', 2), ('bDeviceClass', 1), ('bDeviceSubClass', 1), ('bDeviceProtocol', 1), ('bMaxPacketSize0', 1), ('bNumConfigurations', 1), ('bReserved', 1),]

    def __init__(self, bLength, bDescriptorType, bcdUSB, bDeviceClass, bDeviceSubClass, bDeviceProtocol, bMaxPacketSize0, bNumConfigurations, bReserved, parent_id = None):

        self.parent_id = parent_id
        self.bLength = bLength
       

In [5]:
from sqlalchemy.orm import mapper 

_, _, tables, session = ModelBuilder.get_db_objects(db_url)

class_code = tables[0]
descriptor_type = tables[1]
device_qualifier_descriptor = tables[2]
other_speed_configuration_descriptor = tables[3]
request_code = tables[4]
standard_configuration_descriptor = tables[5]
standard_device_descriptor = tables[6]
standard_endpoint_descriptor = tables[7]
standard_interface_descriptor = tables[8]
string_descriptor_zero = tables[9]
unicode_string_descriptor = tables[10]
usb_device = tables[11]

mapper(ClassCode, class_code)
mapper(DescriptorType, descriptor_type)
mapper(DeviceQualifierDescriptor, device_qualifier_descriptor)
mapper(OtherSpeedConfigurationDescriptor, other_speed_configuration_descriptor)
mapper(RequestCode, request_code)
mapper(StandardConfigurationDescriptor, standard_configuration_descriptor)
mapper(StandardDeviceDescriptor, standard_device_descriptor)
mapper(StandardEndpointDescriptor, standard_endpoint_descriptor)
mapper(StandardInterfaceDescriptor, standard_interface_descriptor)
mapper(StringDescriptorZero, string_descriptor_zero)
mapper(UnicodeStringDescriptor, unicode_string_descriptor)
mapper(UsbDevice, usb_device)

<Mapper at 0x22cb6b3d7b8; UsbDevice>

## Truncate tables

In [6]:
ModelBuilder._truncate_tables(db_url)

## Device

In [7]:
idVendor = 0x1b3f
idProduct = 0x2008

In [8]:
# dev = usb.core.find(idVendor = idVendor, idProduct = idProduct)
dev = universal_serial_bus.USBdevice(vid = idVendor, pid = idProduct)
# print(dev)

In [9]:
usb_device = UsbDevice('GeneralPlus', 'GeneralPlus USB Audio Device')
session.add(usb_device)
session.commit()
usb_device.id

1

## Device descriptor
https://www.beyondlogic.org/usbnutshell/usb5.shtml#DeviceDescriptors  

In [10]:
bmRequestType = usb.util.build_request_type(CONTROL_REQUEST.DIRECTION.IN, 
                                            CONTROL_REQUEST.TYPE.STANDARD, 
                                            CONTROL_REQUEST.RECIPIENT.DEVICE)

In [11]:
descriptor = dev.ctrl_transfer(bmRequestType = bmRequestType, 
                               bRequest = CONTROL_REQUEST.GET_DESCRIPTOR, 
                               wValue = DESCRIPTOR.TYPE.DEVICE << 8, 
                               wIndex = 0, 
                               data_or_wLength = DESCRIPTOR.SIZE.DEVICE)
descriptor

array('B', [18, 1, 16, 1, 0, 0, 0, 8, 63, 27, 8, 32, 0, 1, 1, 2, 0, 1])

In [12]:
dev_dscrpt = StandardDeviceDescriptor.from_byte_array(descriptor, parent_id = usb_device.id)
session.add(dev_dscrpt)
session.commit()
dev_dscrpt.id

1

## Configuration descriptor
https://www.beyondlogic.org/usbnutshell/usb5.shtml#ConfigurationDescriptors  

In [13]:
descriptor = dev.ctrl_transfer(bmRequestType = bmRequestType, 
                               bRequest = CONTROL_REQUEST.GET_DESCRIPTOR, 
                               wValue = DESCRIPTOR.TYPE.CONFIG << 8, 
                               wIndex = 0, 
                               data_or_wLength = DESCRIPTOR.SIZE.CONFIG)
descriptor

array('B', [9, 2, 253, 0, 4, 1, 0, 128, 50])

In [14]:
# config = StandardConfigurationDescriptor(*(StandardConfigurationDescriptor.get_descriptor_fields_values(descriptor)))
config_dscrpt = StandardConfigurationDescriptor.from_byte_array(descriptor, parent_id = dev_dscrpt.id)
session.add(config_dscrpt)
session.commit()
config_dscrpt.id

1

### Host 再一次發送 GetDescriptor(Config類) 的請求
完整的長度 = wTotalLength

In [15]:
descriptor = dev.ctrl_transfer(bmRequestType = bmRequestType, 
                         bRequest = CONTROL_REQUEST.GET_DESCRIPTOR, 
                         wValue = DESCRIPTOR.TYPE.CONFIG << 8, 
                         wIndex = 0, 
                         data_or_wLength = descriptor[2])
# descriptor

In [16]:
descriptors = list(StandardConfigurationDescriptor.split_descriptor(descriptor))
descriptors

[array('B', [9, 2, 253, 0, 4, 1, 0, 128, 50]),
 array('B', [9, 4, 0, 0, 0, 1, 1, 0, 0]),
 array('B', [10, 36, 1, 0, 1, 100, 0, 2, 1, 2]),
 array('B', [12, 36, 2, 1, 1, 1, 0, 2, 3, 0, 0, 0]),
 array('B', [12, 36, 2, 4, 1, 2, 0, 1, 1, 0, 0, 0]),
 array('B', [9, 36, 3, 3, 1, 3, 0, 6, 0]),
 array('B', [9, 36, 3, 2, 1, 1, 0, 9, 0]),
 array('B', [7, 36, 5, 9, 1, 5, 0]),
 array('B', [10, 36, 6, 6, 8, 1, 1, 2, 2, 0]),
 array('B', [9, 36, 6, 5, 4, 1, 67, 0, 0]),
 array('B', [9, 36, 6, 7, 4, 1, 3, 0, 0]),
 array('B', [13, 36, 4, 8, 2, 1, 7, 2, 3, 0, 0, 0, 0]),
 array('B', [9, 4, 1, 0, 0, 1, 2, 0, 0]),
 array('B', [9, 4, 1, 1, 1, 1, 2, 0, 0]),
 array('B', [7, 36, 1, 1, 1, 1, 0]),
 array('B', [14, 36, 2, 1, 2, 2, 16, 2, 68, 172, 0, 128, 187, 0]),
 array('B', [9, 5, 5, 1, 192, 0, 1, 0, 0]),
 array('B', [7, 37, 1, 1, 0, 0, 0]),
 array('B', [9, 4, 2, 0, 0, 1, 2, 0, 0]),
 array('B', [9, 4, 2, 1, 1, 1, 2, 0, 0]),
 array('B', [7, 36, 1, 2, 1, 1, 0]),
 array('B', [14, 36, 2, 1, 1, 2, 16, 2, 68, 172, 0, 1

In [17]:
len(descriptors)

27

## Interface descriptor
https://www.beyondlogic.org/usbnutshell/usb5.shtml#InterfaceDescriptors  

In [18]:
for dscrpt in descriptors:
    if dscrpt[1] == 4:
        dbo = StandardInterfaceDescriptor.from_byte_array(dscrpt, parent_id = config_dscrpt.id)
        session.add(dbo) 
        
session.commit() 

## Endpoint descriptor
https://www.beyondlogic.org/usbnutshell/usb5.shtml#EndpointDescriptors  

In [19]:
for dscrpt in descriptors:
    if dscrpt[1] == 5: 
        dbo = StandardEndpointDescriptor.from_byte_array(dscrpt, parent_id = config_dscrpt.id)
        session.add(dbo) 
        
session.commit() 

## String descriptor
https://www.beyondlogic.org/usbnutshell/usb5.shtml#StringDescriptors  

In [20]:
string_descriptors = dev.get_strings()
string_descriptors

['', 'GeneralPlus', 'USB Audio Device']

In [21]:
for i in range(len(string_descriptors)): 
        if string_descriptors[i] is not None:
            string = UnicodeStringDescriptor(len(string_descriptors[i]),
                                             UnicodeStringDescriptor.int_to_hex(3), 
                                             string_descriptors[i], parent_id = usb_device.id)   
            session.add(string)
        
session.commit() 

## Descriptor types 

In [22]:
from array import array

dataset = [('device', 1), ('configuration', 2), ('string', 3), ('interface', 4), ('endpoint', 5),
           ('device_qualifier', 6), ('other_speed_configuration', 7), ('interface_power', 8),
           ('OTG', 9), ('debug', 10), ('interface_association', 11)]

for row in dataset: 
    do = DescriptorType(row[0],  
                        DescriptorType.int_to_hex(row[1]))
    session.add(do)
        
session.commit() 

## Class code  
https://www.usb.org/defined-class-codes   

In [23]:
dataset = [('Use class information in the Interface Descriptors', 'Device', 0), ('Audio', 'Interface', 1),
           ('Communications and CDC Control', 'Both', 2), ('HID (Human Interface Device)', 'Interface', 3),
           ('Physical', 'Interface', 5), ('Image', 'Interface', 6), ('Printer', 'Interface', 7),
           ('Mass Storage', 'Interface', 8), ('Hub', 'Device', 9), ('CDC-Data', 'Interface', 10),
           ('Smart Card', 'Interface', 11), ('Content Security', 'Interface', 13), ('Video', 'Interface', 14),
           ('Personal Healthcare', 'Interface', 15), ('Audio/Video Devices', 'Interface', 16), 
           ('Billboard Device Class', 'Device', 17), ('USB Type-C Bridge Class', 'Interface', 18),
           ('Diagnostic Device', 'Both', 220), ('Wireless Controller', 'Interface', 224), 
           ('Miscellaneous', 'Both', 239), ('Application Specific', 'Interface', 254), ('Vendor Specific', 'Both', 255), ]

for row in dataset: 
    do = ClassCode(row[0], row[1], 
                   DescriptorType.int_to_hex(row[2]))
    session.add(do)
        
session.commit() 

## Request code  
https://www.beyondlogic.org/usbnutshell/usb6.shtml  

In [24]:
dataset = [('GET_STATUS', 0), ('CLEAR_FEATURE', 1), ('Reserved for future use', 2), ('SET_FEATURE', 3),
           ('Reserved for future use', 4), ('SET_ADDRESS', 5), ('GET_DESCRIPTOR', 6), ('SET_DESCRIPTOR', 7),
           ('GET_CONFIGURATION', 8), ('SET_CONFIGURATION', 9), ('GET_INTERFACE', 10), ('SET_INTERFACE', 11), 
           ('SYNCH_FRAME', 12)]

for row in dataset: 
    do = RequestCode(row[0], 
                   RequestCode.int_to_hex(row[1]))
    session.add(do)
        
session.commit() 